# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [42]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [43]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayacucho,5.6639,-67.6236,91.81,55,76,3.33,VE,2023-10-06 12:52:39
1,1,airuk,7.2772,168.8203,82.74,79,100,15.48,MH,2023-10-06 12:52:39
2,2,mashhad,36.2970,59.6062,57.34,67,40,6.91,IR,2023-10-06 12:52:39
3,3,ushuaia,-54.8000,-68.3000,44.22,56,0,8.01,AR,2023-10-06 12:52:40
4,4,tamanrasset,22.7850,5.5228,85.91,16,40,5.75,DZ,2023-10-06 12:52:40


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [44]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat", 
                                     geo=True, 
                                     size='humidity',
                                     tiles='OSM',  
                                     frame_width=700,
                                     frame_height=500)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [45]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] >= 60) & (city_data_df['Max Temp'] <= 80) & 
    (city_data_df['Cloudiness'] <= 50) & 
    (city_data_df['Humidity'] < 80) &
    (city_data_df['Wind Speed'] <= 15) & (city_data_df['Wind Speed'] >= 5)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,pisco,-13.7000,-76.2167,69.03,77,20,9.22,PE,2023-10-06 12:49:14
18,18,fomboni,-12.2800,43.7425,78.76,77,2,7.23,KM,2023-10-06 12:52:42
19,19,fisterra,42.9049,-9.2629,79.75,70,48,8.50,ES,2023-10-06 12:52:42
25,25,adamstown,-25.0660,-130.1015,68.67,78,31,14.56,PN,2023-10-06 12:48:59
45,45,balaklava,44.5000,33.6000,65.98,49,16,12.75,UA,2023-10-06 12:52:46


### Step 3: Create a new DataFrame called `hotel_df`.

In [46]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
7,7,pisco,-13.7000,-76.2167,69.03,77,20,9.22,PE,2023-10-06 12:49:14,
18,18,fomboni,-12.2800,43.7425,78.76,77,2,7.23,KM,2023-10-06 12:52:42,
19,19,fisterra,42.9049,-9.2629,79.75,70,48,8.50,ES,2023-10-06 12:52:42,
25,25,adamstown,-25.0660,-130.1015,68.67,78,31,14.56,PN,2023-10-06 12:48:59,
45,45,balaklava,44.5000,33.6000,65.98,49,16,12.75,UA,2023-10-06 12:52:46,
52,52,flagstaff,35.1981,-111.6513,73.60,30,0,8.05,US,2023-10-06 12:50:53,
55,55,carnarvon,-24.8667,113.6333,68.47,79,0,13.58,AU,2023-10-06 12:52:47,
67,67,hami,42.8000,93.4500,66.83,21,1,5.93,CN,2023-10-06 12:52:49,
83,83,taltal,-25.4000,-70.4833,61.27,65,46,7.14,CL,2023-10-06 12:52:51,
120,120,meadow lake,34.8014,-106.5436,74.14,43,18,7.20,US,2023-10-06 12:52:58,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [54]:
# Set parameters to search for a hotel
radius = 1000 #meters


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params = {
        'categories': 'accommodation.hotel',  # Filter for hotels
        'filter': f'circle:{longitude},{latitude},{radius}',  # Circle filter with center at lat, long and radius
        'bias': f'proximity:{longitude},{latitude}',  # Bias towards proximity
        'limit': 1,  
        'apiKey': geoapify_key  
    }
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df
hotel_df = hotel_df.drop('No hotel found')

Starting hotel search
pisco - nearest hotel: No hotel found
fomboni - nearest hotel: No hotel found
fisterra - nearest hotel: Hotel Temático do Banco Azul
adamstown - nearest hotel: No hotel found
balaklava - nearest hotel: Рыбацкая слобода
flagstaff - nearest hotel: Residence Inn
carnarvon - nearest hotel: No hotel found
hami - nearest hotel: No hotel found
taltal - nearest hotel: Hostería Taltal
meadow lake - nearest hotel: No hotel found
fredonia - nearest hotel: The White Inn
nouadhibou - nearest hotel: Hotel valencia
christiana - nearest hotel: No hotel found
nadi - nearest hotel: President Hotel
valparaiso - nearest hotel: Ibis Hotel
giado - nearest hotel: No hotel found
kas - nearest hotel: KEKOVA OTEL
luanda - nearest hotel: Hotel Forum
porto san giorgio - nearest hotel: Hotel Garden
tobruk - nearest hotel: فندق زهرة البطنان -طبرق
praia da vitoria - nearest hotel: Salles
la passe - nearest hotel: No hotel found
mahina - nearest hotel: No hotel found
tralee - nearest hotel: The 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [63]:
hotel_df = hotel_df[hotel_df['Hotel Name'] != 'No hotel found']

hotel_df
    

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
19,19,fisterra,42.9049,-9.2629,79.75,70,48,8.50,ES,2023-10-06 12:52:42,Hotel Temático do Banco Azul
45,45,balaklava,44.5000,33.6000,65.98,49,16,12.75,UA,2023-10-06 12:52:46,Рыбацкая слобода
52,52,flagstaff,35.1981,-111.6513,73.60,30,0,8.05,US,2023-10-06 12:50:53,Residence Inn
83,83,taltal,-25.4000,-70.4833,61.27,65,46,7.14,CL,2023-10-06 12:52:51,Hostería Taltal
121,121,fredonia,42.4401,-79.3317,71.06,71,40,10.36,US,2023-10-06 12:52:58,The White Inn
122,122,nouadhibou,20.9310,-17.0347,72.64,78,5,14.52,MR,2023-10-06 12:52:58,Hotel valencia
138,138,nadi,-17.8000,177.4167,64.40,77,20,5.75,FJ,2023-10-06 12:50:23,President Hotel
179,179,valparaiso,-33.0393,-71.6273,63.00,61,40,9.22,CL,2023-10-06 12:53:10,Ibis Hotel
185,185,kas,36.2018,29.6377,74.59,64,11,7.20,TR,2023-10-06 12:53:11,KEKOVA OTEL
187,187,luanda,-8.8368,13.2343,78.80,78,40,10.36,AO,2023-10-06 12:48:49,Hotel Forum


In [65]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points("Lng", "Lat", 
                                     geo=True, 
                                     size='humidity',
                                     tiles='OSM',
                                     hover_cols=['Hotel Name','City','Country'],  
                                     frame_width=700,
                                     frame_height=500)

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,City,Country)